In [10]:
# 📦 Install dependencies
!pip install transformers torch datasets accelerate seqeval rouge-score nltk --quiet

In [11]:
import torch
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
print("Using device:", "CUDA" if device == 0 else "CPU")

Using device: CPU


In [12]:
# Sentiment Analysis
sentiment_pipe = pipeline("sentiment-analysis",
                          model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                          device=device)

# Named Entity Recognition (NER)
ner_pipe = pipeline("ner",
                    model="dslim/bert-base-NER",
                    aggregation_strategy="simple",
                    device=device)

# Question Answering (QA)
qa_pipe = pipeline("question-answering",
                   model="distilbert-base-cased-distilled-squad",
                   device=device)

# Summarization
summ_pipe = pipeline("summarization",
                     model="facebook/bart-large-cnn",
                     device=device)

print("✅ All pipelines loaded successfully!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoin

✅ All pipelines loaded successfully!


In [13]:
import gradio as gr

def nlp_task_selector(task, text, question=None):
    if not text.strip():
        return "⚠️ Please enter some text."

    # SENTIMENT
    if task == "Sentiment Classification":
        result = sentiment_pipe(text)[0]
        return f"🧩 Sentiment: {result['label']} ({result['score']:.2f})"

    # NER
    elif task == "Named Entity Recognition":
        entities = ner_pipe(text)
        if not entities:
            return "No named entities found."
        return "\n".join([f"{e['word']} → {e['entity_group']} ({e['score']:.2f})" for e in entities])

    # QUESTION ANSWERING
    elif task == "Question Answering":
        if not question.strip():
            return "⚠️ Please enter a question."
        result = qa_pipe(question=question, context=text)
        return f"💬 Answer: {result['answer']} ({result['score']:.2f})"

    # SUMMARIZATION
    elif task == "Summarization":
        summary = summ_pipe(text, max_length=80, min_length=30, do_sample=False)[0]['summary_text']
        return f"📰 Summary:\n{summary}"

    else:
        return "Invalid task selected."

In [14]:
task_options = ["Sentiment Classification", "Named Entity Recognition", "Question Answering", "Summarization"]

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("## 🧠 BERT for NLP Tasks (Interactive Demo)")
    gr.Markdown("Select a task below and input your text. Uses pre-trained BERT-based models from Hugging Face.")

    task = gr.Radio(task_options, label="Choose NLP Task")
    text = gr.Textbox(label="Enter Text / Context", lines=5, placeholder="Type or paste your text here...")
    question = gr.Textbox(label="Enter Question (only for QA task)", placeholder="e.g. Who invented transformers?")
    output = gr.Textbox(label="Result", lines=5)

    btn = gr.Button("Run Task 🚀")
    btn.click(nlp_task_selector, inputs=[task, text, question], outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c7164602208b3ead5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Summary

This notebook demonstrates how BERT-based Transformer models can perform key NLP tasks such as Sentiment Classification, Named Entity Recognition (NER), Question Answering (QA), and Text Summarization using PyTorch and Hugging Face Transformers.

It also uses Gradio to build an interactive interface, allowing users to enter text, select a task, and instantly see model predictions — all without writing additional code.

Overall, this lab shows how pre-trained Transformer models can be easily applied for real-world language understanding and generation tasks.